In [ ]:
%pip install flask-ngrok
%pip install pyngrok
%ngrok authtoken 2Sy2LWPQ2xqvJOk65EY8RMwsEKA_6NfruRjiBmV7s6BFhkUap
%pip install speechrecognition
%ip install gradio TTS
from flask import Flask, request
import speech_recognition as sr

app = Flask(__name__)

class Transcribe_File:
    def __init__(self):
        self.recognizer = sr.Recognizer()

    def transcribe_audio(self, audio_file):
        with sr.AudioFile(audio_file) as source:
            audio_data = self.recognizer.record(source)
            try:
                text = self.recognizer.recognize_google(audio_data, language="lo-LA")
                return {"transcription": text}
            except sr.UnknownValueError:
                return {"error": "Google Speech Recognition could not understand the audio"}
            except sr.RequestError as e:
                return {"error": f"Could not request results from Google Speech Recognition service; {e}"}

In [ ]:
from flask import Flask
from flask_ngrok import run_with_ngrok
from distutils.log import error
import mimetypes
from re import template
from urllib import request
import io
import os
from flask import render_template, request, send_file
import tempfile
import gradio as gr
import numpy as np
import os
import json
from typing import Optional
from TTS.config import load_config
from TTS.utils.manage import ModelManager
from TTS.utils.synthesizer import Synthesizer

model_path = "D:\Downloads\project final\final_model\checkpoint_100000.pth" #path of model_path
config_path = "D:\Downloads\project final\final_model\config.json" #path pf config_path
speakers_file_path = "D:\Downloads\project final\final_model\language_ids.json" #path of speakers_file_path

speaker_idx = "Nick"

synthesizer = Synthesizer(
    model_path, config_path, speakers_file_path, None, None, None,
)
PEOPLE_FOLDER = os.path.join('D:\Downloads\project final\website_with_flask', 'templates')
app = Flask(__name__, template_folder="D:\Downloads\project final\website_with_flask\templates", static_folder="D:\Downloads\project final\website_with_flask\templates")
app.config['UPLOAD_FOLDER'] = PEOPLE_FOLDER
run_with_ngrok(app)
@app.route("/")

def index():
    full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'LoiyToiyDev.png')
    return render_template("index.html",user_image = full_filename)

@app.route("/call/martha", methods = ["POST"])
def call_martha():
    if not request.form["text"]:
        return {"error": "Please provide the text"}, 400
    text = request.form["text"]
    outputs = synthesizer.tts(text, speaker_idx,language_name="lo-la")
    out = io.BytesIO()
    synthesizer.save_wav(outputs, out)
    return send_file(out, mimetype="audio/wav")

@app.route("/transcribe", methods=["POST"])
def transcribe():
    transcriber = Transcribe_File()
    audio_file = request.files["audio_file"]
    if audio_file:
        result = transcriber.transcribe_audio(audio_file)
        return result
    else:
        return {"error": "No audio file provided"}
app.run()